<h1>Automated Flood Detection based on Satellite Images</h1>
<br>
This project focuses on flood detection using only the SEN2 data from the SEN12-FLOOD dataset. The goal is to process Sentinel-2 satellite images, specifically four spectral bands: Band2 (blue), Band3 (green), Band4 (red), and Band8 (infrared). These bands are important for distinguishing between land, water, and other surface features. The process involves organizing the data, checking for empty images, and stacking the bands together for further analysis, which will be used for flood detection tasks.
<br>
<hr>
Dataset:<br>
Clément Rambour, Nicolas Audebert, Elise Koeniguer, Bertrand Le Saux, Michel Crucianu, Mihai Datcu. (2020). SEN12-FLOOD : a SAR and Multispectral Dataset for Flood Detection . IEEE Dataport. https://dx.doi.org/10.21227/w6xz-s898

Download (after free registration) - 12,2Gb: https://ieee-dataport.org/open-access/sen12-flood-sar-and-multispectral-dataset-flood-detection


In [6]:
import rasterio
import pandas as pd
import os
import shutil
import cv2

<h1>Functions</h1>
<ul>
<li>check_empty_img(url)</li>
<li>remove_folders(path)</li>
<li>create_dir_structure(flist)</li>
<li>stack_bands(path)</li>
</ul>

In [7]:
def check_empty_img(url):
    
    path = os.path.join(url, 'B02.tif')

# Load the image in grayscale mode (0)
    image = cv2.imread(path,0)
  
 # Return True if all pixels are zero (empty image), otherwise False
    if (cv2.countNonZero(image) == 0):
        return  True
    else:
        return  False
  

In [8]:
def remove_folders(path):
    # Delete the folder and all its contents
    shutil.rmtree(path)
    return

In [18]:
def create_dir_structure(flist):
    for folder in flist:
        prefix = folder[:13]    # Extract prefix from folder name
        id = folder[13:]        # Extract id from folder name

        if len(id) > 4: continue # Skip if id length is more than 4, because the id can be at most 4 digits

        for file in os.listdir(folder):

            if file.startswith('S1'): 
                os.remove(os.path.join(prefix + id, file)) # Remove files starting with 'S1'
                continue
            # If the file is a spectral band (B02, B03, B04, B08)
            if file.endswith('B02.tif') or file.endswith('B03.tif') or file.endswith('B04.tif') or file.endswith('B08.tif'):
                date = file[3:13] # Extract date from file name
                newFolder = prefix + 'S2_' + id + '_' +date # Create new folder name

                if os.path.isdir(newFolder):
                    shutil.move(os.path.join(folder, file), os.path.join(newFolder, file[14:])) # Move file if folder exists
                else:
                    os.mkdir(newFolder)     # Create folder if it doesn't exist
                    shutil.move(os.path.join(folder, file), os.path.join(newFolder, file[14:]))  # Move file

        remove_folders(folder) # Remove the original folder after moving files


In [19]:


def stack_bands(path):
    print(path)

    
    band_list = ['B02.tif', 'B03.tif', 'B04.tif', 'B08.tif']
    try:
        
        # Read metadata from the first band (B02.tif) 
        with rasterio.open(os.path.join(path, band_list[0])) as src0:
            meta = src0.meta # Extract metadata from the first band
        
        # Update metadata to reflect the number of bands (layers) 
        meta.update(count = len(band_list))

       # Create a new stack file and write each band to it
        with rasterio.open(os.path.join(path, 'stack.tif'), 'w', **meta) as dst:
            for id, layer in enumerate(band_list, start=1):
                with rasterio.open(os.path.join(path, layer)) as src1:
                    dst.write_band(id, src1.read(1)) # Write each band to the stack file
    except:
        print("Folder with no Data") # Handle cases where the folder has no valid data 
        remove_folders(path) # Remove the folder if there's an issue
        
        pass # Continue without stopping on error 

    

<h2> Data operations </h2>

In [28]:
# Create a list of all folders containing spectral bands


flist = []

rootdir = os.path.normpath('./SEN12FLOOD')  # Get the absolute path for the root directory
#rootdir = './SEN12FLOOD' 
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file) # Create the full path for each file/folder
    if os.path.isdir(d):    # Check if it's a folder
        flist.append(d)     # Add folder to the list
        
# Print the total number of folders found        
print(f"The number of folders are currently = {len(flist)}")

The number of folders are currently = 2237


In [24]:
# Call the function to create the directory structure for the initial folder list (can run up to 1-2 min)
create_dir_structure(flist)


In [29]:
# Create a new list of folders that start with 'S2_'

flist = []
#rootdir = './SEN12FLOOD'
rootdir = os.path.normpath('./SEN12FLOOD')
for file in os.listdir(rootdir):
     if 'S2_' in file:
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            flist.append(d)
        
        
print(f"The number of folders are currently = {len(flist)}")


The number of folders are currently = 2237


In [30]:
# Iterate through all folders and create a new image with 4 spectral bands:
# Band2 (blue), Band3 (green), Band4 (red), and Band8 (infrared)
#This process can take up to 2-5 minutes to complete


for folder_path in flist:
    empty = check_empty_img(folder_path) # Check if the folder contains empty images (all pixels are zero)
    if empty:
        
        print("The images inside the current folder are empty - zero")
        remove_folders(folder_path) # Remove the folder if images are empty 
    else:
        stack_bands(folder_path) # Stack the spectral bands if images are valid

    
    

SEN12FLOOD\00S2_01_2018-12-18
SEN12FLOOD\00S2_01_2018-12-23
SEN12FLOOD\00S2_01_2019-01-02
SEN12FLOOD\00S2_01_2019-01-12
SEN12FLOOD\00S2_01_2019-01-17
SEN12FLOOD\00S2_01_2019-01-22
SEN12FLOOD\00S2_01_2019-01-27
SEN12FLOOD\00S2_01_2019-02-01
SEN12FLOOD\00S2_01_2019-02-06
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_04_2018-12-17
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_04_2018-12-22
The images inside the current folder are empty - zero
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_04_2019-01-06
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_04_2019-01-16
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_05_2018-12-17
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_05_2018-12-22
The images inside the current folder are empty - zero
The images inside the current folder are empty - zero
SEN12FLOOD\00S2_05_2019-01-06
The images inside the current fold